<a href="https://colab.research.google.com/github/PRaliphada/ExpectedLoss/blob/main/Sentiment_Data_From_NewsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [7]:
import requests
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Your NewsAPI key
newsapi_key = '6dc23737eb7144c5b3d46c55ed9e6f1b'

# Financial entities to search for
entities = ['Facebook', 'Amazon', 'Microsoft', 'Google']

# Set up VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)['compound']
    if sentiment_score >= 0.05:
        return 'Positive'
    elif sentiment_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Function to get news articles
def get_news(entity, api_key, num_articles=100):
    url = f'https://newsapi.org/v2/everything?q={entity}&apiKey={api_key}&language=en&pageSize={num_articles}'
    response = requests.get(url)
    data = response.json()

    news_data = []
    if data['status'] == 'ok':
        articles = data['articles']
        for article in articles:
            news_content = article['description'] if article['description'] else article['content']
            sentiment = get_sentiment(news_content)
            news_data.append([article['source']['name'], entity, sentiment, news_content])

    return news_data

# Store all news data
all_news = []

for entity in entities:
    all_news.extend(get_news(entity, newsapi_key))

# Create a DataFrame
df = pd.DataFrame(all_news, columns=['Source', 'Entity', 'Sentiment', 'News Content'])

# Save to CSV
df.to_csv('financial_news_sentiment.csv', index=False)

print(df)


                  Source    Entity Sentiment  \
0    Yahoo Entertainment  Facebook  Negative   
1    Yahoo Entertainment  Facebook  Positive   
2              The Verge  Facebook  Positive   
3               BBC News  Facebook   Neutral   
4            Gizmodo.com  Facebook  Positive   
..                   ...       ...       ...   
395            [Removed]    Google   Neutral   
396            [Removed]    Google   Neutral   
397          Gizmodo.com    Google   Neutral   
398          Gizmodo.com    Google  Negative   
399         Slashdot.org    Google  Positive   

                                          News Content  
0    Meta’s latest round of account takedowns inclu...  
1    Congress can’t get enough of the Meta boss. Ni...  
2    Meta is shutting down the Spark API, which let...  
3    Nicola Smith booked the caravan on Facebook, b...  
4    The Facebook founder really seems to want Repu...  
..                                                 ...  
395                     

In [8]:
article_count = df['Entity'].value_counts()
print(article_count)

Entity
Facebook     100
Amazon       100
Microsoft    100
Google       100
Name: count, dtype: int64
